In [38]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD

In [39]:
from sklearn.preprocessing import LabelEncoder
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [27]:
n_comp = 10

# tSVD
#tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
#tsvd_results_train = tsvd.fit_transform(train.drop(["y"], axis=1))
#tsvd_results_test = tsvd.transform(test)

# PCA
#pca = PCA(n_components=n_comp, random_state=420)
#pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
#pca2_results_test = pca.transform(test)

# ICA
#ica = FastICA(n_components=n_comp, random_state=420)
#ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
#ica2_results_test = ica.transform(test)

# GRP
#grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
#grp_results_train = grp.fit_transform(train.drop(["y"], axis=1))
#grp_results_test = grp.transform(test)

# SRP
#srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
#srp_results_train = srp.fit_transform(train.drop(["y"], axis=1))
#srp_results_test = srp.transform(test)

In [28]:
#usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
#for i in range(1, n_comp + 1):
    #train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    #test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    #train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    #test['ica_' + str(i)] = ica2_results_test[:, i - 1]

   # train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    #test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    #train['grp_' + str(i)] = grp_results_train[:, i - 1]
    #test['grp_' + str(i)] = grp_results_test[:, i - 1]

    #train['srp_' + str(i)] = srp_results_train[:, i - 1]
    #test['srp_' + str(i)] = srp_results_test[:, i - 1]

In [40]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y','ID'], axis=1).values
X_test= test.drop(['ID'], axis=1).values

In [42]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

5

In [43]:
for train_index, test_index in kf.split(X):
    X_train, y_train=X[train_index], y[train_index]
    X_valid, y_valid=X[test_index], y[test_index]

In [44]:
forest= RandomForestRegressor(n_estimators=100, random_state=0, max_depth=5)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [45]:
from sklearn.metrics import r2_score
print("R^2 valid: {: }".format(r2_score(y_valid, forest.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, forest.predict(X_train))))

R^2 valid:  0.5911715362591088
R^2 train:  0.6025906693231765


In [34]:
y_pred=forest.predict(X_test)

In [35]:
sub=pd.DataFrame()

In [36]:
sub['ID']=id_test
sub["y"]=y_pred

In [37]:
sub.to_csv("stackforest_pca+ica10.csv", index=False)